<a href="https://colab.research.google.com/github/Talha1818/Machine-Learning-Python-Notebooks/blob/master/Check_Packages_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import requests
import json
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
!pip install PyPDF4~=1.27.0
!pip install pdf2image~=1.16.0
!sudo apt install tesseract-ocr
!pip install pytesseract
!sudo apt-get install poppler-utils
#!pip install paddleocr
#!pip install paddlepaddle paddleocr
#!pip install Pillow==9.0.0 --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 KB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyPDF4: filename=PyPDF4-1.27.0-py3-none-any.whl size=61246 sha256=7d5d655982f77b0952c9b3c7b02672a781c722198f21794d888060f9ca8074f6
  Stored in directory: /root/.cache/pip/wheels/cd/0e/4a/e6f842a6035ccffff0dab29c39dd06c3427560a82783355a83
Successfully built PyPDF4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 23 not upgraded.
Need to get 4,850 kB of archives.
After this ope

In [ ]:
import csv
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
from PyPDF4.pdf import BytesIO
import os
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import clear_output 
import io
#from paddleocr import PaddleOCR,draw_ocr
#from PIL import Image
#ocr = PaddleOCR(use_angle_cls=True, lang='en', use_space_char = True, gpu_mem = 16000, use_gpu=True)


In [ ]:
FOLDER_PATH = '/content/drive/MyDrive/Annotation Folder/Few_Packages'
PDF_PATH = "PDF"
CSV_PATH = "CSV"

In [ ]:
# # Map the images from folder to form a DataFrame
# def get_all_images_from_subdirectory_to_dataframe(path, extension='.csv'):
#   configfiles = [os.path.join(dirpath, f)
#       for dirpath, dirnames, files in os.walk(path)
#       for f in files if f.endswith(f'{extension}')] 
#   images_list = [(i) for i in tqdm(configfiles)]
#   return images_list

In [ ]:
# Map the images from folder to form a DataFrame
def get_all_images_from_subdirectory_to_dataframe(path, extension='.csv'):
  configfiles = [os.path.join(dirpath, f)
      for dirpath, dirnames, files in os.walk(path)
      for f in files if f.endswith(f'{extension}')] 
  images_list = [(i.split("/")[-1].split(".")[0],i.split("/")[-1].split("_")[1],i.split("/")[-1].split("_")[2].split(".")[0], i) for i in tqdm(configfiles)]
  return images_list

In [ ]:
# l = get_all_images_from_subdirectory_to_dataframe(FOLDER_PATH+"/"+PDF_PATH, extension='.pdf')
# pdf_files = pd.DataFrame(data=l, columns=['package_path'])
# pdf_files.head()

In [ ]:
l = get_all_images_from_subdirectory_to_dataframe(FOLDER_PATH+"/"+PDF_PATH, extension='.pdf')
pdf_files = pd.DataFrame(data=l, columns=['package_name','state', 'county', 'package_path'])
pdf_files.head()

  0%|          | 0/7 [00:00<?, ?it/s]

,package_name,state,county,package_path
0,Raheem_SAMPLE_CHECKING,SAMPLE,CHECKING,/content/drive/MyDrive/Annotation Folder/Few_P...
1,Sample_9_NJ,9,NJ,/content/drive/MyDrive/Annotation Folder/Few_P...
2,117665_NJ_CAMDEN,NJ,CAMDEN,/content/drive/MyDrive/Annotation Folder/Few_P...
3,218246_NJ_SALEM,NJ,SALEM,/content/drive/MyDrive/Annotation Folder/Few_P...
4,218248_NJ_CAMDEN,NJ,CAMDEN,/content/drive/MyDrive/Annotation Folder/Few_P...


In [ ]:
l = get_all_images_from_subdirectory_to_dataframe(FOLDER_PATH+"/"+CSV_PATH, extension='.csv')
csv_files = pd.DataFrame(data=l, columns=['package_name','state', 'county', 'package_path'])
csv_files.head()

  0%|          | 0/7 [00:00<?, ?it/s]

,package_name,state,county,package_path
0,Raheem_SAMPLE_CHECKING,SAMPLE,CHECKING,/content/drive/MyDrive/Annotation Folder/Few_P...
1,Sample_9_NJ,9,NJ,/content/drive/MyDrive/Annotation Folder/Few_P...
2,218245_NJ_CAMDEN,NJ,CAMDEN,/content/drive/MyDrive/Annotation Folder/Few_P...
3,218246_NJ_SALEM,NJ,SALEM,/content/drive/MyDrive/Annotation Folder/Few_P...
4,218248_NJ_CAMDEN,NJ,CAMDEN,/content/drive/MyDrive/Annotation Folder/Few_P...


In [ ]:
pdf_files['package_path'].iloc[0].split("/")[-1]

'Raheem_SAMPLE_CHECKING.pdf'

In [ ]:
def get_images_from_file(filename):
    try:
        return convert_from_path(filename, thread_count=8)
    except:
        return []

def get_text_from_image(image):        
    page_text = ""
    with BytesIO() as f:
            image.save(f, format='png')
            f.seek(0)
            ima_jpg = Image.open(f, formats=['png'])
            page_text = pytesseract.image_to_string(ima_jpg, 
                                                    config=r'-l eng --psm 6')
    return page_text

In [ ]:
def save_data(file):
    # Open the PDF file in read-binary mode
    with open(file, 'rb') as pdf_file:  
        images = get_images_from_file(file)

        # print(len(images)) 
        dic = {}
        dic['page#'] = []
        dic['package_name'] = []
        dic['text'] = []
        # dic['token'] = []
        get_package_name = file.split("/")[-1]
        for page_num in tqdm(range(0, len(images))):
            try:
              dic['page#'].append(page_num+1)
              dic['package_name'].append(get_package_name)
              dic['text'].append(get_text_from_image(images[page_num]))
              # dic['token'].append(token)        
            except:
              print(f"Issue in file {get_package_name} PAGE### ({page_num+1})")
              dic['page#'].append('')
              dic['package_name'].append('')
              dic['text'].append('')
        df = pd.DataFrame(dic)
        return df.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)


In [ ]:
def save_data_PDF_CSV(file):
    # Open the PDF file in read-binary mode
    with open(file, 'rb') as pdf_file:    
        get_package_name = file.split("/")[-1].split(".")[0]
        try:
          get_csv_path = csv_files[csv_files['package_name']==get_package_name]['package_path'].values[0]
          # print(get_csv_path)
        except FileNotFoundError:
          print(f"This csv {get_package_name} package not found...")

        ########################### Read CSV FILE ###########################################
        # Read CSV file into a string
        with open(get_csv_path, 'r') as f:
            data = f.read()

        # Determine whether the file is comma-separated or tab-separated
        num_commas = data.split('\n')[0].count(',')
        num_tabs = data.split('\n')[0].count('\t')
        if num_commas > num_tabs:
            # File is comma-separated, so do nothing
            modified_data_csv = data
        else:
            # File is tab-separated, so replace tabs with commas
            modified_data_csv = data.replace('\t', ',')
        LOAD_CSV_FILE = pd.read_csv(io.StringIO(modified_data_csv))
        ############################################################################################
        # print(LOAD_CSV_FILE)
        images = get_images_from_file(file)

        # print(len(images)) 
        dic = {}
        dic['page#'] = []
        dic['package_name'] = []
        dic['text'] = []
        dic['token'] = []

        for page_num in tqdm(range(0, len(images))):
            try:
              token = LOAD_CSV_FILE[LOAD_CSV_FILE['No. of Pages']==page_num+1]['Token'].values.tolist()[0]
              # print(page_num+1, "==>", token)
              dic['page#'].append(page_num+1)
              dic['package_name'].append(get_package_name)
              dic['text'].append(get_text_from_image(images[page_num]))
              dic['token'].append(token)        
            except:
              print(f"Issue in file {get_package_name} PAGE### ({page_num+1})")
              dic['page#'].append('')
              dic['package_name'].append('')
              dic['text'].append('')
              dic['token'].append('')
        df = pd.DataFrame(dic)
        return df.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)


In [ ]:
import multiprocessing

# Get the total number of available CPU cores
num_cores = multiprocessing.cpu_count()

# Print the total number of available CPU cores
print(f"Total number of available CPU cores: {num_cores}")

Total number of available CPU cores: 40


In [ ]:
def process_file(file):
    get_package_name = file.split("/")[-1].split(".")[0]
    print(get_package_name)
    DATA = save_data_PDF_CSV(file)
    DATA.to_csv(f"/content/{get_package_name}.csv", index=False)

if __name__ == '__main__':
    with multiprocessing.Pool(processes=6) as pool:
        tqdm(pool.map(process_file, tqdm(pdf_files['package_path'])))

  0%|          | 0/7 [00:00<?, ?it/s]

Raheem_SAMPLE_CHECKINGSample_9_NJ117665_NJ_CAMDEN218246_NJ_SALEM218248_NJ_CAMDEN117664_NJ_CAMDEN







  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/85 [00:00<?, ?it/s]

...

  0%|          | 0/99 [00:00<?, ?it/s]

Issue in file 117664_NJ_CAMDEN PAGE### (1)
Issue in file 117664_NJ_CAMDEN PAGE### (2)
Issue in file 117664_NJ_CAMDEN PAGE### (3)
Issue in file 117664_NJ_CAMDEN PAGE### (4)
Issue in file 117664_NJ_CAMDEN PAGE### (5)
Issue in file 117664_NJ_CAMDEN PAGE### (6)
Issue in file 117664_NJ_CAMDEN PAGE### (7)
Issue in file 117664_NJ_CAMDEN PAGE### (8)
Issue in file 117664_NJ_CAMDEN PAGE### (9)
Issue in file 117664_NJ_CAMDEN PAGE### (10)
Issue in file 117664_NJ_CAMDEN PAGE### (11)
Issue in file 117664_NJ_CAMDEN PAGE### (12)
Issue in file 117664_NJ_CAMDEN PAGE### (13)
Issue in file 117664_NJ_CAMDEN PAGE### (14)
Issue in file 117664_NJ_CAMDEN PAGE### (15)
Issue in file 117664_NJ_CAMDEN PAGE### (16)
Issue in file 117664_NJ_CAMDEN PAGE### (17)
Issue in file 117664_NJ_CAMDEN PAGE### (18)
Issue in file 117664_NJ_CAMDEN PAGE### (19)
Issue in file 117664_NJ_CAMDEN PAGE### (20)
Issue in file 117664_NJ_CAMDEN PAGE### (21)
Issue in file 117664_NJ_CAMDEN PAGE### (22)
Issue in file 117664_NJ_CAMDEN PAGE### (2

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
import glob

folder_path = "/content"
file_pattern = "*.csv"

csv_files__ = glob.glob(os.path.join(folder_path, file_pattern))
PACKAGE_NAME = [i.split("/")[-1].split(".")[0] for i in csv_files__]
print("Total Packages: ", len(PACKAGE_NAME))

Total Packages:  8


In [ ]:
dataframe = []
for i in tqdm(csv_files__):
  dataframe.append(pd.read_csv(i))

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
df = pd.concat(dataframe, axis = 0)
df.shape

(775, 5)

In [ ]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

# Define regex patterns
punct_pattern = r'[^\w\s]'
num_pattern = r'\d+'
special_pattern = r'[^A-Za-z0-9\s]'

# Define cleaning function
def clean_text(text):
    text = re.sub(r'\\n|\n', ' ', text)
    # remove links
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'\x00|\\x00', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(punct_pattern, '', text)
    # Remove integers
    text = re.sub(num_pattern, '', text)
    # Remove special characters
    text = re.sub(special_pattern, '', text)
    # Remove extra whitespace
    text = re.sub('\s+', ' ', text).strip()
    # Remove nextline
    text = re.sub(r'\n', ' ', text)
    # Remove words with length 1
    text = ' '.join(word for word in text.split() if len(word) > 2)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

In [ ]:
df.head()

,page#,package_name,text,token,our_prediction
0,1.0,Raheem_SAMPLE_CHECKING,American Land Title Association Commitment for...,1.0,NaN
1,2.0,Raheem_SAMPLE_CHECKING,American Land Title Association Commitment for...,1.0,NaN
2,3.0,Raheem_SAMPLE_CHECKING,American Land Title Association Commitment for...,1.0,NaN
3,4.0,Raheem_SAMPLE_CHECKING,American Land Title Association Commitment for...,1.0,NaN
4,5.0,Raheem_SAMPLE_CHECKING,American Land Title Association Commitment for...,1.0,NaN


In [ ]:
df = df[df['text'].notna()]

In [ ]:
df.to_csv("/content/drive/MyDrive/Annotation Folder/Few_Packages/Sample8Packages.csv", index=False)

In [ ]:
df.head()

,page#,package_name,text,token,our_prediction
0,1.0,Raheem_SAMPLE_CHECKING,American Land Title Association Commitment for...,1.0,NaN
1,2.0,Raheem_SAMPLE_CHECKING,American Land Title Association Commitment for...,1.0,NaN
2,3.0,Raheem_SAMPLE_CHECKING,American Land Title Association Commitment for...,1.0,NaN
3,4.0,Raheem_SAMPLE_CHECKING,American Land Title Association Commitment for...,1.0,NaN
4,5.0,Raheem_SAMPLE_CHECKING,American Land Title Association Commitment for...,1.0,NaN


In [ ]:
df['package_name'].value_counts()

Raheem_SAMPLE_CHECKING    105
Sample_9_NJ               102
218245_NJ_CAMDEN          100
218246_NJ_SALEM            92
218248_NJ_CAMDEN           90
117665_NJ_CAMDEN           85
Name: package_name, dtype: int64

In [ ]:
# df[df['package_name'].isin(['Raheem_SAMPLE_CHECKING', ''])]

In [ ]:
# df = df[df['package_name']=='Sample_9_NJ']

In [ ]:
df.iloc[0]

page#                                                           1
package_name                                          Sample_9_NJ
text            File No.: CAHD-612050\nAFFIDAVIT OF TITLE\nPUR...
token                                                           3
Name: 0, dtype: object

In [ ]:
df['token'].value_counts()

30    19
20    15
1      8
40     7
14     5
2      5
12     4
15     4
43     3
41     3
42     3
37     2
24     2
35     2
3      2
11     2
44     2
23     2
13     2
38     1
22     1
39     1
27     1
21     1
28     1
25     1
36     1
29     1
45     1
Name: token, dtype: int64

# **PREDICTION**

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Annotation Folder/Few_Packages/Sample8Packages.csv")
df = df[~df['package_name'].isin(['Raheem_SAMPLE_CHECKING', 'Sample_9_NJ'])]
df.head()

,page#,package_name,text,token,our_prediction
105,1.0,218248_NJ_CAMDEN,File No.: CAHD-612066\nAFFIDAVIT OF TITLE\nPUR...,3.0,NaN
106,2.0,218248_NJ_CAMDEN,| File No.: CAHD-612066\n| 7. Exceptions and A...,3.0,NaN
107,3.0,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Buyer's Settlement Sta...",11.0,NaN
108,4.0,218248_NJ_CAMDEN,Description\na Credit\nTitle - Lender's NJRB 5...,11.0,NaN
109,5.0,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Seller's Settlement St...",13.0,NaN


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

new_model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Annotation Folder/BertModelFiles/BERT_MODEL_FULL_SAMPLE_GREATER_100')
new_tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Annotation Folder/BertModelFiles/tokenizer')

# new_model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/BertModelFiles/BERT_MODEL')
# new_tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/BertModelFiles/tokenizer')


# new_model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Annotation Folder/Category-I Model/CategoryI')
# new_tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Annotation Folder/Category-I Model/tokenizer')

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

actual_label = {0: 0, 1: 1, 2: 2, 3: 3, 4: 11,
                5: 12, 6: 13, 7: 14, 8: 15, 9: 20, 
                10: 21, 11: 22, 12: 23, 13: 24, 14: 25, 
                15: 27, 16: 28, 17: 29, 18: 30, 19: 32, 20: 33,
                21: 34, 22: 35, 23: 36, 24: 37, 25: 39, 26: 40, 27: 41, 28: 42, 29: 43, 
                30: 44, 31: 45, 32: 46, 33: 49, 34: 50, 35: 51, 36: 52, 37: 53, 38: 54, 39: 55, 
                40: 62, 41: 63, 42: 64, 43: 65, 44: 66, 45: 67, 46: 68, 47: 69, 48: 70, 49: 71, 50: 72,
                51: 73, 52: 75, 53: 76, 54: 81, 55: 90, 56: 93, 57: 94, 58: 95, 59: 96, 60: 98, 61: 99, 
                62: 100, 63: 111, 64: 112, 65: 113, 66: 118, 67: 123, 68: 133, 69: 153, 70: 156, 71: 165, 
                72: 166, 73: 168, 74: 170, 75: 185, 76: 214, 77: 222, 78: 223, 79: 225, 80: 239, 81: 242, 
                82: 288, 83: 289, 84: 322, 85: 325, 86: 333, 87: 348, 88: 390, 89: 448, 90: 451, 91: 521, 92: 927}

actual_label = {k: str(v) for k, v in actual_label.items()}

In [ ]:
import psutil


PACKAGE_PREDICTION = []
def get_prediction(text):
    encoding = new_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512,
                      add_special_tokens=True, # Add [CLS] and [SEP] tokens
                      )
    encoding = {k: v for k,v in encoding.items()}

    # Get initial CPU and RAM usage
    process = psutil.Process()
    initial_cpu = process.cpu_times()

    initial_ram = process.memory_info().rss / 1e6

    outputs = new_model(**encoding)
    

    logits = outputs.logits
  
    probs = F.softmax(logits, dim=1).squeeze().cpu().detach().numpy()
    

    label = np.argmax(probs, axis=-1)

    # Get CPU and RAM usage after running the model
    cpu_usage = sum(process.cpu_times()) - sum(initial_cpu)
    ram_usage = (process.memory_info().rss / 1e6) - initial_ram
    result = {
        'label': int(actual_label[label]),
        'probability': max(probs),
        'cpu_usage (seconds)': cpu_usage,
        'ram_usage (MB)': ram_usage
    }
    PACKAGE_PREDICTION.append(result)
    return result, logits

In [ ]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
import re


# Define regex patterns
punct_pattern = r'[^\w\s]'
num_pattern = r'\d+'
special_pattern = r'[^A-Za-z0-9\s]'

# Define cleaning function
def clean_text(text):
    text = re.sub(r'\\n|\n', ' ', text)
    # remove links
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'\x00|\\x00', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(punct_pattern, '', text)
    # Remove integers
    text = re.sub(num_pattern, '', text)
    # Remove special characters
    text = re.sub(special_pattern, '', text)
    # Remove extra whitespace
    text = re.sub('\s+', ' ', text).strip()
    # Remove nextline
    text = re.sub(r'\n', ' ', text)
    # Remove words with length 1
    text = ' '.join(word for word in text.split() if len(word) > 2)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# df1 = pd.read_csv("/content/drive/MyDrive/Annotation Folder/Category-I Model/testing.csv")
# df1.head()

In [ ]:
# df2 = df1[df1['package_name']=='612201_PA_PHILADELPHIA'].sort_values(by='page#')
# df2

In [ ]:
from tqdm import tqdm
prediction = []
for pck, text, token, pg in tqdm(df[['package_name', 'text', 'token', 'page#']].values):
  prepare_text = "<START> " + text + " <END> <DOCUMENT>"
  input_text = clean_text(prepare_text)
  res = get_prediction(input_text)
  res['page#'] = pg
  res['package_name'] = pck
  res['text'] = text
  res['token'] = token
  prediction.append(res)

  0%|          | 0/367 [00:00<?, ?it/s]


TypeError: ignored

In [ ]:
df_res = pd.DataFrame(prediction)

In [ ]:
df_res.columns

Index([], dtype='object')

In [ ]:
df_res = df_res[['page#','package_name', 'text', 'token','label', 'probability', 
                 'cpu_usage (seconds)','ram_usage (MB)']]

df_res['page#'] = df_res['page#'].astype(int)

KeyError: ignored

In [ ]:
df_res.head()

,page#,package_name,text,token,label,probability,cpu_usage (seconds),ram_usage (MB)
0,1,218248_NJ_CAMDEN,File No.: CAHD-612066\nAFFIDAVIT OF TITLE\nPUR...,3.0,3,0.999406,6.12,535.719936
1,2,218248_NJ_CAMDEN,| File No.: CAHD-612066\n| 7. Exceptions and A...,3.0,3,0.999380,3.83,47.271936
2,3,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Buyer's Settlement Sta...",11.0,11,0.999020,3.84,566.349824
3,4,218248_NJ_CAMDEN,Description\na Credit\nTitle - Lender's NJRB 5...,11.0,11,0.999001,3.49,137.326592
4,5,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Seller's Settlement St...",13.0,13,0.998211,3.69,149.762048


In [ ]:
df_sa = df_res[['label', 'token']].rename({'label':'our_prediction'}, axis = 1)

In [ ]:
df_sa.to_csv("Result.csv", index=False)

In [ ]:
# !pip install tabulate
from tabulate import tabulate
print(tabulate(df_res[['label', 'token']], headers = 'keys', tablefmt = 'psql'))

In [ ]:
df_res['if_probability_greater_80_then_prediction'] = np.where(df_res['probability']>0.8, 'PREDICTED', 'NOT-PREDICTED')

In [ ]:
df_res.columns

Index(['page#', 'package_name', 'text', 'token', 'label', 'probability',
       'cpu_usage (seconds)', 'ram_usage (MB)',
       'if_probability_greater_80_then_prediction'],
      dtype='object')

In [ ]:
df_res = df_res[['page#','package_name', 'text','token', 'label', 'probability','if_probability_greater_80_then_prediction', 'cpu_usage (seconds)',
       'ram_usage (MB)']]

df_res['token']  =  df_res['token'].astype(int)  
df_res.head()

,page#,package_name,text,token,label,probability,if_probability_greater_80_then_prediction,cpu_usage (seconds),ram_usage (MB)
0,1,218248_NJ_CAMDEN,File No.: CAHD-612066\nAFFIDAVIT OF TITLE\nPUR...,3,3,0.999406,PREDICTED,4.67,102.182912
1,2,218248_NJ_CAMDEN,| File No.: CAHD-612066\n| 7. Exceptions and A...,3,3,0.999380,PREDICTED,4.64,36.196352
2,3,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Buyer's Settlement Sta...",11,11,0.999020,PREDICTED,4.59,102.453248
3,4,218248_NJ_CAMDEN,Description\na Credit\nTitle - Lender's NJRB 5...,11,11,0.999001,PREDICTED,4.70,102.453248
4,5,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Seller's Settlement St...",13,13,0.998211,PREDICTED,4.64,101.642240


In [ ]:
# df_res[df_res['if_probability_greater_80_then_prediction']=='NOT-PREDICTED']

In [ ]:
df_token = pd.read_csv("/content/drive/MyDrive/Annotation Folder/lender_docs_token.csv")
df_token.head()

,Keys,Token,Meaning,Classification
0,CS,0.0,Cover Sheet,Lender Docs
1,MUC,1.0,MARKED UP COMMITMENT,NaN
2,DC,2.0,DEED COPY,NaN
3,AOT,3.0,AFF OF TITLE,NaN
4,IA,4.0,ID AFFIDAVIT (Buyer),NaN


In [ ]:
def get_token_meaning(df, target_name = 'target'):
  token_name = []
  for i in df[target_name]:
    try:
      token_name.append(df_token[df_token['Token']==i]['Meaning'].values[0])
    except:
      token_name.append('')
  return token_name

In [ ]:
token_name = []
for i in df_res['label']:
  try:
    token_name.append(df_token[df_token['Token']==i]['Meaning'].values[0])
  except:
    token_name.append('')

In [ ]:
df_res['Meaning'] = token_name

In [ ]:
df_res.head()

,page#,package_name,text,token,label,probability,if_probability_greater_80_then_prediction,cpu_usage (seconds),ram_usage (MB),Meaning
0,1,218248_NJ_CAMDEN,File No.: CAHD-612066\nAFFIDAVIT OF TITLE\nPUR...,3,3,0.999406,PREDICTED,4.67,102.182912,AFF OF TITLE
1,2,218248_NJ_CAMDEN,| File No.: CAHD-612066\n| 7. Exceptions and A...,3,3,0.999380,PREDICTED,4.64,36.196352,AFF OF TITLE
2,3,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Buyer's Settlement Sta...",11,11,0.999020,PREDICTED,4.59,102.453248,ALTA BUYER’S STATEMENT
3,4,218248_NJ_CAMDEN,Description\na Credit\nTitle - Lender's NJRB 5...,11,11,0.999001,PREDICTED,4.70,102.453248,ALTA BUYER’S STATEMENT
4,5,218248_NJ_CAMDEN,"City Abstract, LLC ALTA Seller's Settlement St...",13,13,0.998211,PREDICTED,4.64,101.642240,ALTA SELLER’S STATEMENT


In [ ]:
df_res.columns

Index(['page#', 'package_name', 'text', 'token', 'label', 'probability',
       'if_probability_greater_80_then_prediction', 'cpu_usage (seconds)',
       'ram_usage (MB)', 'Meaning'],
      dtype='object')

In [ ]:
df_res_ = df_res[['page#','package_name','token', 'label', 'probability',
       'if_probability_greater_80_then_prediction', 'Meaning']]

df_res_['probability'] = (df_res_['probability'] * 100).round(2).astype(str) + "%"

<ipython-input-83-e4da06c51d0b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_res_['probability'] = (df_res_['probability'] * 100).round(2).astype(str) + "%"


In [ ]:
df_res_ = df_res_.rename({'label':'our_prediction_token', 'token':'actual_token'}, axis = 1)
df_res_.head()

,page#,package_name,actual_token,our_prediction_token,probability,if_probability_greater_80_then_prediction,Meaning
0,1,218248_NJ_CAMDEN,3,3,99.94%,PREDICTED,AFF OF TITLE
1,2,218248_NJ_CAMDEN,3,3,99.94%,PREDICTED,AFF OF TITLE
2,3,218248_NJ_CAMDEN,11,11,99.9%,PREDICTED,ALTA BUYER’S STATEMENT
3,4,218248_NJ_CAMDEN,11,11,99.9%,PREDICTED,ALTA BUYER’S STATEMENT
4,5,218248_NJ_CAMDEN,13,13,99.82%,PREDICTED,ALTA SELLER’S STATEMENT


In [ ]:
df_res_['package_name'].value_counts()

218245_NJ_CAMDEN    100
218246_NJ_SALEM      92
218248_NJ_CAMDEN     90
117665_NJ_CAMDEN     85
Name: package_name, dtype: int64

In [ ]:
from sklearn.metrics import accuracy_score
acc1 = accuracy_score(df_res_[df_res_['package_name']=='218245_NJ_CAMDEN']['actual_token'], 
                      df_res_[df_res_['package_name']=='218245_NJ_CAMDEN']['our_prediction_token'])

acc2 = accuracy_score(df_res_[df_res_['package_name']=='218246_NJ_SALEM']['actual_token'], 
                      df_res_[df_res_['package_name']=='218246_NJ_SALEM']['our_prediction_token'])

acc3 = accuracy_score(df_res_[df_res_['package_name']=='218248_NJ_CAMDEN']['actual_token'], 
                      df_res_[df_res_['package_name']=='218248_NJ_CAMDEN']['our_prediction_token'])

acc4 = accuracy_score(df_res_[df_res_['package_name']=='117665_NJ_CAMDEN']['actual_token'], 
                      df_res_[df_res_['package_name']=='117665_NJ_CAMDEN']['our_prediction_token'])

print("Average Accuray: ", np.mean([acc1, acc2, acc3, acc4]).round(4)*100, "%")

Average Accuray:  96.99 %


In [ ]:
df_res_.to_csv("/content/drive/MyDrive/Annotation Folder/Few_Packages/TestingPackagesResults.csv", index=False)
print("SAVED!!!")

SAVED!!!


In [ ]:
filename = 'TestingPackagesResults.csv'
Folder_name = '/content/drive/MyDrive/Annotation Folder/Few_Packages'

In [ ]:
def get_app_password(filename):
    with open(filename, 'r') as openfile:
        credentials = json.load(openfile)
    password = credentials['password']
    return password

In [ ]:
password = get_app_password("/content/credentials - Copy.json")

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# Set up email parameters
email_user = "muhammadtalha1818@gmail.com"
email_password = password
email_send = "meet.ahmad247@gmail.com"
subject = "Sample Documents Prediction Attachment"

body = "Sample Documents Prediction"

# Create a multipart message object and set headers
msg = MIMEMultipart()
msg['From'] = email_user
msg['To'] = email_send
msg['Subject'] = subject

# Attach body to message
msg.attach(MIMEText(body, 'plain'))

# Open the file in bynary
attachment = open(Folder_name+'/'+filename, "rb")

# Create a base64 encoded attachment object
part = MIMEBase('application', 'octet-stream')
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

# Attach the file to the message object and send the email
msg.attach(part)
text = msg.as_string()
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(email_user, email_password)

server.sendmail(email_user, email_send, text)
server.quit()
print(f">>> S E N T to {email_send}")

>>> S E N T to meet.ahmad247@gmail.com


In [ ]:
df_p = pd.DataFrame(PACKAGE_PREDICTION)[['label', 'probability']]

In [ ]:
# !pip install tabulate
from tabulate import tabulate


In [ ]:
print(tabulate(df_p, headers = 'keys', tablefmt = 'psql'))

# **Embedding New Algo**

In [ ]:
# df_p.loc[3, 'label'] = 30
# df_p.loc[3, 'probability'] = 0.75

In [ ]:
def checkSinglePackage(df):
  df1 = df['label'].value_counts().to_frame().reset_index().rename(columns={'index':'label','label':'count'})
  one_category = df1[df1['count']==1]
  one_category_list= one_category['label'].tolist()

  if len(one_category_list)>0:
    print("Single Token: ", one_category_list)
  else:
    print("No single Token present in package.")
  return one_category_list

In [ ]:
one_category_list = checkSinglePackage(df_p)

No single Token present in package.


In [ ]:
def passDataToNewAlgoDesign(df, thresh=0.8, one_category_list=[]):
  s = df.values.tolist()
  FLAG = False

  THRESHOLD = thresh
  for i in range(len(s)-1):
      if s[i][0] in one_category_list:
          pass
      elif s[i][0]==s[i+1][0]:
          pass
      else:
          if (s[i][1]<THRESHOLD or s[i+1][1]<THRESHOLD) and not (s[i][0] in one_category_list or s[i+1][0] in one_category_list):
              prev_ = s[i][0]
              prev1 = s[i-1][0]
              next_ = s[i+1][0]
              next1 = s[i+2][0]
              next2 = s[i+3][0]
              # print(f"Index:{i}  Previous1 {prev1} | Previous {prev_} Prob {s[i][1]} \t| Next {next_} Prob {s[i+1][1]} | Next1 {next1} | Next2 {next2}")
              if (prev_==next1 and prev_==next2) and (prev_==prev1):
                  print("="*100)
                  print(f"\t\tWe need to set {prev1} at INDEX {i+1}")
                  print("="*100)              
                  df.loc[i+1,'label'] = prev1
                  FLAG = True

  if FLAG==False:
    print("Everything is fine.")
  return df.values.tolist()

In [ ]:
df_new = passDataToNewAlgoDesign(df_p, 0.8, one_category_list)

Everything is fine.


In [ ]:
df_new

[[3.0, 0.9994056224822998],
 [3.0, 0.9993802309036255],
 [11.0, 0.999019980430603],
 [11.0, 0.9990013241767883],
 [13.0, 0.998211145401001],
 [13.0, 0.9912443161010742],
 [14.0, 0.9980692267417908],
 [14.0, 0.9978371262550354],
 [14.0, 0.9981209635734558],
 [14.0, 0.9981722831726074],
 [14.0, 0.9981209635734558],
 [2.0, 0.9974607229232788],
 [2.0, 0.996867835521698],
 [2.0, 0.99603670835495],
 [2.0, 0.8805390000343323],
 [23.0, 0.9996376037597656],
 [23.0, 0.9996287822723389],
 [29.0, 0.9996126294136047],
 [40.0, 0.9996241331100464],
 [40.0, 0.9975518584251404],
 [40.0, 0.9961761236190796],
 [40.0, 0.9969965219497681],
 [40.0, 0.9991487264633179],
 [40.0, 0.9975882768630981],
 [40.0, 0.9984952211380005],
 [40.0, 0.9974182844161987],
 [36.0, 0.9991384744644165],
 [44.0, 0.9992402791976929],
 [44.0, 0.9991753697395325],
 [44.0, 0.9992455244064331],
 [22.0, 0.9994125366210938],
 [43.0, 0.9930468201637268],
 [43.0, 0.9848967790603638],
 [43.0, 0.9929980039596558],
 [41.0, 0.994007468223571

# **Get Encoding**

In [ ]:
text = """NEW JERSEY LAND TITLE                                       NJRB 3-09
INSURANCE RATING BUREAU                               Last Revised: 09/01/19


American Land Title Association                                                    Commitment for Title Insurance 
                                                                                                Adopted 08-01-2016 
                                                                                 Technical Corrections 04-02-2018


                                               SCHEDULE B, PART 11 
                                                     Exceptions

THIS COMMITMENT DOES NOT REPUBLISH ANY COVENANT, CONDITION, RESTRICTION, OR LIMITATION 
CONTAINED IN ANY DOCUMENT REFERRED TO IN THIS COMMITMENT TO THE EXTENT THAT THE 
SPECIFIC COVENANT, CONDITION, RESTRICTION, OR LIMITATION VIOLATES STATE OR FEDERAL LAW 
BASED ON RACE, COLOR, RELIGION, SEX, SEXUAL ORIENTATION, GENDER IDENTITY, HANDICAP, 
FAMILIAL STATUS, OR NATIONAL ORIGIN.

The Policy will not insure against loss or damage resulting from the terms and provisions of any lease or easement 
identified in Schedule A, and will include the following Exceptions unless cleared to the satisfaction of the Company:

    Notwithstanding any provision of the policy to the contrary, any encroachment, encumbrance, violation, 
variation or adverse circumstance affecting the Title that would be disclosed by an accurate and a complete land 
survey of the Land.       no surve

 .  Any defect, lien, encumbrance, adverse claim, or other matter that appears for the first time in the Public
 ecords or is created, attaches, or is disclosed between the Commitment Date and the date on which all of the 
 chedule B, Part I-Requirements are met.

 .  Easements, or claims of easements, not shown by the public records.

 .  Rights or claims by parties in possession not shown by the public records

    Any lien, or right to a lien, for services, labor or material heretofore or hereafter furnished, imposed by law and 
  t shown by the public records.
6.  Taxes, municipal charges and assessments as set forth in the searches enclosed4th quarter taxes paid

7.  Subsurface conditions and/or encroachments not disclosed by an instrumeWo1 i.e—cWd718vTir4P?icy only) 

8.  Subject to possible added or omitted assessments as provided by N.J.S.A 54:4-63.1 et seq.

9.  Rights, public and private, in and to all road, streets and avenues crossing, bounding or affecting the 
pre   ises.

10.   Rights, public and private, together with flooding and drainage rights, if any, in and to all streams, rivers, or 
wat ar courses crossing, bounding or affecting the premises.

11.   Policy will not insure acreage or quantity of land.

12.   Vacant or abandoned properties registration fee or fine or penalty due and owing to the municipality, if 
applicable.

13.   Easement as contained in Book 910 and Page 377.

14.   Declaration of Covenants and Restrictions as contained in Book 3891 and Page 608.

15.   Subject to all terms, notes, conditions, setbacks, easements, and restrictions as shown and/or set forth on 
thatcertain map filed as filed map #348-7.

16.   Subject to 25 foot front setback line as shown on Filed Map No. 348-7.

This page is only a part of a 2016 ALTA® Commitment for Title Insurance issued by Old Republic National Title Insurance Company. This 
Commitment is not valid without the Notice; the Commitment to Issue Policy; the Commitment Conditions; Schedule A; Schedule B, Part (- 
Requirements; Schedule B, Part II—Exceptions; and a counter-signature by the Company or its issuing agent that may be in electronic form. """

prepare_text = "<START> " + text + " <END> <DOCUMENT>"
input_text = clean_text(prepare_text)
input_text

'start new jersey land title njrb insurance rating bureau last revised american land title association commitment title insurance adopted technical corrections schedule part exceptions commitment republish covenant condition restriction limitation contained document referred commitment extent specific covenant condition restriction limitation violates state federal law based race color religion sex sexual orientation gender identity handicap familial status national origin policy insure loss damage resulting terms provisions lease easement identified schedule include following exceptions unless cleared satisfaction company notwithstanding provision policy contrary encroachment encumbrance violation variation adverse circumstance affecting title would disclosed accurate complete land survey land surve defect lien encumbrance adverse claim matter appears first time public ecords created attaches disclosed commitment date date chedule part irequirements met easements claims easements show

In [ ]:
def get_embeddings(text):
    encoding = new_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=512,
                      add_special_tokens=True, # Add [CLS] and [SEP] tokens
                      )
    encoding = {k: v for k,v in encoding.items()}
    return new_model(**encoding).logits[0].squeeze().detach().numpy()

In [ ]:
get_embeddings("Hi how are you man?")

array([ 1.8816991 ,  0.4658825 ,  2.0105612 ,  1.9459666 , -1.4426985 ,
       10.861579  ,  3.4298263 ,  3.756084  ,  1.8135382 , -1.6893377 ,
        0.09273741, -0.4012715 , -1.1657249 , -0.89559263,  1.0865326 ,
       -1.8711303 ,  3.295951  , -1.6996468 ,  0.48856533, -1.5788653 ,
       -1.5610192 , -1.4114034 ,  0.10628329, -2.5829587 , -0.74501055,
       -1.7452567 ,  3.1905565 , -1.6492747 , -1.4256879 , -1.207754  ,
       -0.48824707, -5.0260186 , -1.2722965 , -0.8361792 , -2.180726  ,
       -2.026762  , -4.376067  , -1.0666806 , -2.3282628 , -0.4394204 ,
       -5.639468  , -4.703514  , -2.2531068 ,  0.9443826 , -2.5210435 ,
       -2.8121274 , -0.69964874, -0.21445863, -2.975415  , -5.085027  ,
       -0.56816316, -1.4427944 , -4.184082  , -1.8958902 , -2.7402778 ,
       -2.8972719 , -0.5309164 , -2.1144671 , -1.83762   , -5.5997114 ,
       -0.8787428 , -1.1285644 , -1.4770752 , -3.1397932 , -1.3619322 ,
       -0.47526896, -2.2052634 , -2.786751  , -2.4831076 , -1.90

In [ ]:
a = [[3.0, 0.9994056224822998],
 [3.0, 0.9993802309036255],
 [11.0, 0.999019980430603],
 [11.0, 0.9990013241767883]]

In [ ]:
import pandas as pd

In [ ]:
df_final = pd.DataFrame(df_new, columns=['label', 'probability', 'cpu_usage (seconds)', 'ram_usage (MB)']).to_dict(orient='records')

[{'label': 3.0, 'prob': 0.9994056224822998},
 {'label': 3.0, 'prob': 0.9993802309036255},
 {'label': 11.0, 'prob': 0.999019980430603},
 {'label': 11.0, 'prob': 0.9990013241767883}]